# **Tarea 1**
## Machine Learning Avanzado
Integrantes: Patricio Ramirez
             Carlos Bustamante
             Nicolas Rivera
             Pablo Elgueta

---

##### Objetivo:
En este laboratorio volveremos a ver la biblioteca del trimestre anterior de perros y gatos. la base de datos de imagenes se
encuentra en la siguiente dirección: ‘https://www.kaggle.com/competitions/dogs-vs-cats/data‘
Ademas, para completar el análisis, se le pide construir una red que permita hacer la clasificación en base a los audios
adicionalmente: ‘https://www.kaggle.com/datasets/mmoreaux/audio-cats-and-dogs‘ 

###### Se le evaluará por:

- Carga y lectura de imagenes
- Crear una red neuronal convolucional
- Crear una red neuronal LSTM
- Pruebe 2 formas distintas de arquitectura de red para cada una.
- Cargue dos fotos de su biblioteca (sí, vaya y saque una foto a un perro o gato).
- Evalúe sus modelos con las fotos sacadas.
- Cargue los datos de audio
- Proponga un modelo para clasificar este tipo de dato
- Evalúe su modelo.

### Dogs vs Cats

#### **Descripción Dataset**


- El archivo de entrenamiento contiene 25.000 imagenes de perros y gatos. 
- Entrena tu algoritmo con estos archivos y predice las etiquetas para test1.zip 

#### **Etiquetas:**

- 1 = dog 
- 0 = cat


## 1.-Reconocimiento e importación de librerías



Para manipular bases de datos se usó Pandas, para visualización fueron las librerías Matplotlib y Seaborn, y en cuanto a Machine Learning se utilizó Keras y Scikit-learn.

In [ ]:
import opendatasets as od

#Analisis de Datos

import pandas as pd
import math
import numpy as np
import random

#Visualizacion de Datos

import matplotlib.pyplot as plt
import seaborn as sns

#Machine Learning

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import models, layers, optimizers, regularizers
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tqdm import tqdm
sns.set(rc={'figure.figsize':(11.7,8.27)})


#test = pd.read_csv("/home/pato/Magister Data Science/Tercer Trimestre Data Science/ML Avanzado/Tareas/titanic/test.csv")
#train = pd.read_csv("/home/pato/Magister Data Science/Tercer Trimestre Data Science/ML Avanzado/Tareas/titanic/train.csv")

In [ ]:
# Nuevas

from matplotlib.image import imread
from matplotlib import pyplot

## 1.1.-Cargando el Dataset

In [1]:
od.download("https://www.kaggle.com/competitions/dogs-vs-cats/data")
{"username":"dgrone","key":"8e16739c70ce24ea1e6d9a45b61e9796"}

NameError: name 'od' is not defined

In [ ]:
def extract_zip(file, folder, password):
    import zipfile
    ruta_zip = file
    ruta_extraccion = folder
    archivo_zip = zipfile.ZipFile(ruta_zip, 'r')
    try:
        archivo_zip.extractall(pwd=password, path=ruta_extraccion)
    except:
        pass
    archivo_zip.close()

In [ ]:
from os import listdir
folder = 'dogs-vs-cats/train/'
if 'train' not in  listdir('dogs-vs-cats/'):
    file = 'dogs-vs-cats/train.zip'
    password = None
    folder = 'dogs-vs-cats'
    extract_zip(file, folder, password)
    folder = 'dogs-vs-cats/train/'

from os import listdir
folder = 'dogs-vs-cats/train/'

i = 0
j = 0
k = 0
for i in range(len(listdir(folder))):
    if listdir(folder)[i].startswith('cat'):
        j = j+1
        print(j)
    if listdir(folder)[i].startswith('dog'):
        k = k+1
    
print(f'cats images = {j}')
print(f'cdogs images = {k}')

In [ ]:
for i in range(3):
    pyplot.subplot(130  + 1 + i)
    filename = folder + 'dog.' + str(i) + '.jpg'
    image = imread(filename)
    pyplot.imshow(image)
pyplot.show()

for i in range(3):
    pyplot.subplot(130  + 1 + i)
    filename = folder + 'cat.' + str(i) + '.jpg'
    image = imread(filename)
    pyplot.imshow(image)
pyplot.show()

In [ ]:
#Generaremos un subset de Perros para poder probar con menos tiempo de procesamiento
from tensorflow.keras.utils import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
from numpy import asarray

i=0
photos = list()
labels = list()
for image in range(12500):   # en versión final se debe reemplazar por file in listdir(folder), hay que probar
    file = 'dog.'+ str(i) + '.jpg'
    photo = load_img(folder + file, target_size=(50, 50))
    photo = img_to_array(photo)
    photos.append(photo)
    if file.startswith('dog'):
        label = 1.0
        labels.append(label)
    file = 'cat.'+ str(i) + '.jpg'
    photo = load_img(folder + file, target_size=(50, 50))
    photo = img_to_array(photo)
    photos.append(photo)
    if file.startswith('cat'):
        label = 0.0
        labels.append(label)
photos = asarray(photos)
labels = asarray(labels)
print(photos.shape, labels.shape)


In [ ]:
photos[1]

In [ ]:

from numpy import save
from numpy import load
save('dogs_vs_cats_photos.npy', photos)
save('dogs_vs_cats_labels.npy', labels)
photos = load('dogs_vs_cats_photos.npy')
labels = load('dogs_vs_cats_labels.npy')
print(photos.shape, labels.shape)

In [ ]:
photos = photos/255

In [ ]:
photos[2]

In [ ]:
input_shape = photos[1].shape
num_classes = 1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(photos, labels,test_size=0.3,random_state=12)

import tensorflow

## Primero, el modelo es secuencial
model = keras.models.Sequential()

## agregamos capa convolucional 2D
model.add(
    tensorflow.keras.layers.Conv2D(
        32,
        kernel_size=(3,3),
        activation='relu',
        input_shape = input_shape
    )
)

## agregamos segunda capa convolucional 2D
model.add(
    tensorflow.keras.layers.Conv2D(
        64,
        kernel_size=(3,3),
        activation='relu'
    )
)

## agregamos capa de Pooling
model.add(
    tensorflow.keras.layers.MaxPooling2D(
        pool_size=(2,2)
    )
)

## agregamos capa de regularizacion Dropout para minimizar calculos
model.add(
    tensorflow.keras.layers.Dropout(
        0.25
    )
)

## agregamos capa de aplanado
model.add(
    tensorflow.keras.layers.Flatten())

## agregamos capa de conexion completa
model.add(
    tensorflow.keras.layers.Dense(
        units=128,
        activation='relu'
    )
)

## agregamos capa de regularizacion Dropout para minimizar calculos
model.add(
    tensorflow.keras.layers.Dropout(
        0.5
    )
)

## agregamos capa de conexion completa
model.add(
    tensorflow.keras.layers.Dense(
        units=num_classes,
        activation='softmax'
    )
)


model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer = keras.optimizers.Adadelta(),
             metrics=['accuracy'])

In [ ]:
batch_size = 128
epochs=20 

model.fit(X_train,y_train,batch_size=batch_size,epochs=epochs,verbose = 1,validation_data=(X_test,y_test))

score = model.evaluate(X_test,y_test,verbose = 0)
print('Test loss: ',score[0])
print('Test accuract: ',score[1])

In [ ]:
# Modelo más Preciso

model = keras.models.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(tf.keras.layers.Conv2D(64,(3,3), activation = 'relu', input_shape = input_shape))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))
# Add another:
model.add(tf.keras.layers.Conv2D(64,(3,3), activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


In [ ]:
# Compilación Mejor Accuracy

model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose = 1, validation_data=(X_test,y_test) )
score = model.evaluate(X_test,y_test,verbose = 0)
print('Test loss: ',score[0])
print('Test accuract: ',score[1])


# load dogs vs cats dataset, reshape and save to a new file





from numpy import save

photos, labels = list(), list()
# enumerate files in the directory
for file in listdir(folder):
	# determine class
	output = 0.0

	labels.append(output)
# convert to a numpy arrays


print(photos.shape, labels.shape)
# save the reshaped photos
save('dogs_vs_cats_photos.npy', photos)
save('dogs_vs_cats_labels.npy', labels)

In [ ]:
modelCNN = keras.models.Sequential()

modeloCNN = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')])

In [ ]:
modeloCNN.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
modeloCNN.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose = 1, validation_data=(X_test,y_test) )
score = model.evaluate(X_test,y_test,verbose = 0)
print('Test loss: ',score[0])
print('Test accuract: ',score[1])

In [ ]:
modelCNN2 = keras.models.Sequential()

modeloCNN2 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(250, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')])

In [ ]:
# Compilación Mejor Accuracy

modeloCNN2.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
modeloCNN2.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose = 1, validation_data=(X_test,y_test) )
score = model.evaluate(X_test,y_test,verbose = 0)
print('Test loss: ',score[0])
print('Test accuract: ',score[1])

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

model = keras.models.Sequential()

tensorboardCNN2 = TensorBoard(log_dir='logs/cnn2')
modeloCNN2.fit(X_train, y_train, batch_size=32,
                validation_split=0.15,
                epochs=12,
                callbacks=[tensorboardCNN2])

In [ ]:
modelCNN3 = keras.models.Sequential()

modeloCNN3 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(250, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')])

In [ ]:
# Compilación Mejor Accuracy

modeloCNN3.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# define our loss function and optimizer
modeloCNN3.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

# train the parameters
history = modeloCNN3.fit(X_train, y_train, epochs=epochs, batch_size=16)

# evaluate accuracy
train_acc = modeloCNN3.evaluate(X_train, y_train, batch_size=16)[1]
test_acc = modeloCNN3.evaluate(X_test, y_test, batch_size=16)[1]
print('Training accuracy: %s' % train_acc)
print('Testing accuracy: %s' % test_acc)

losses = history.history['loss']

plt.plot(range(len(losses)), losses, 'r')
plt.show()

## 1.2.-Procesamiento de datos y análisis descriptivo

a) Se observa una distribución de edades similar entre hombres y mujeres, sin embargo, los hombres poseen los registros con las edades mas grandes.

b) La cantidad de hombres a bordo casi duplica a la de mujeres.

c) 342 personas sobrevivieron, correspondientes al 38%.

d) El 74% de mujeres sobrevivió, mientras que en los hombres solo un 19% lo hizo.

In [ ]:
train[['Survived','PassengerId','Sex']].groupby(['Sex','Survived']).agg(['count'])

In [ ]:
#Representación de Heatmap todas las variables contínuas
ax.set_title("Heatmap considerando sólo variables continuas")
corrMatrix_cont = train[['Age','SibSp', 'Parch', 'Fare']].corr('pearson')
sns.heatmap(corrMatrix_cont, annot=True)

No se aprecian correlaciones fuertes, siendo la mas significativa la de hermanos/parejas con padres e hijos, sugiriendo que algunos pasajeros viajaban en familia.

In [ ]:
train.info()

Al comprobar los valores nulos usando la función info, es posible detectar que la variable **Age** tiene 177 registros vacíos, **Cabin** tiene 687 y **Embarked** tiene 2.

En **Age**, los nulos serán rellenados con la edad promedio de la muestra, en **Cabin**, los valores nulos representan a las personas que no iban en una pieza particular, y en **Embarked** fueron rellenados con la moda.

In [ ]:
#Quitamos columnas con informacion unica
train.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True, )

#Binarizar columna Cabin
train["Cabin"] = train["Cabin"].astype(str) != 'nan'
train["Cabin"] = train["Cabin"].astype(int)

#Rellenando los NA de la edad con la media
median_age = train['Age'].median()
train['Age'].fillna(median_age, inplace=True)
train['Age']=train['Age'].astype(int)

#Se rellenan 2 na de Embarked con la moda (S)
embarked_mode = train['Embarked'].mode()
train['Embarked'].fillna(embarked_mode.values[0], inplace=True)

train.info()

In [ ]:
sns.histplot(x='Age', data=train, hue='Survived')
#Se crea una variable categorica para la edad con el proposito de separar a las personas menores a 14 anos
train['Age_cat'] = pd.cut(train['Age'], bins=[-1, 14, float('Inf')], labels=['kid', 'adult'])

El análisis de la distribución de la edad en los supervivientes permite observar que las personas menores de 14 años son mas propensas a sobrevivir, es por eso que se genera una variable categórica para agruparlas.

In [ ]:
sns.histplot(x='Fare', data=train, hue='Survived')
#Se puede crear también una variable categórica para las tarifas mayores a 50.
train['Fare_cat'] = pd.cut(train['Fare'], bins=[-1, 35, float('Inf')], labels=['cheap', 'expensive'])

Ocurre una situación similar con las tarifas, por eso se agruparon entre menores y mayores a $35.

In [ ]:
train[['Pclass', 'Survived']].groupby(['Pclass']).agg(['mean', 'count'])

In [ ]:
train[['Embarked', 'Pclass', 'Survived']].groupby(['Embarked', 'Pclass']).agg(['mean', 'count'])

In [ ]:
#Binarización de las variables categóricas
train["Sex"] = train['Sex'].replace(['female','male'],[0,1])
train["Sex"] = train["Sex"].astype(int)
Age_dum = pd.get_dummies(train['Age_cat'], prefix = "age")
Fare_dum = pd.get_dummies(train['Fare_cat'], prefix = "fare")
Embarked_dum = pd.get_dummies(train['Embarked'], prefix = "embarked")
Pclass_dum = pd.get_dummies(train['Pclass'], prefix = "pclass")
train.head()
#Añadimos las nuevas variables, ahora como dummies, al dataset
train = pd.concat([train, Embarked_dum, Age_dum, Fare_dum, Pclass_dum], axis = 1)
#Se borran también algunos dummies para asegurar invertibilidad de la matríz 
train = train.drop(columns = ['Pclass','Age', 'Fare', 'Embarked',"Age_cat","age_adult", "Fare_cat","embarked_C","fare_expensive",'pclass_1'])
train.head()

In [ ]:
#Aplicamos las mismas operaciones sobre la data de testeo
#Tratamiento NaNs y columnas nuevas
test.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True, )
test["Cabin"] = test["Cabin"].astype(str) != 'nan'
test["Cabin"] = test["Cabin"].astype(int)
median_age2 = test['Age'].median()
test['Age'].fillna(median_age2, inplace=True)
test['Age']=test['Age'].astype(int)
embarked_mode2 = test['Embarked'].mode()
test['Embarked'].fillna(embarked_mode2.values[0], inplace=True)
test['Age_cat'] = pd.cut(test['Age'], bins=[-1, 14, float('Inf')], labels=['kid', 'adult'])
test['Fare_cat'] = pd.cut(test['Fare'], bins=[-1, 35, float('Inf')], labels=['cheap', 'expensive'])

#Binarización de las variables categóricas
test["Sex"] = test['Sex'].replace(['female','male'],[0,1])
test["Sex"] = test["Sex"].astype(int)
Age_dum2 = pd.get_dummies(test['Age_cat'], prefix = "age")
Fare_dum2 = pd.get_dummies(test['Fare_cat'], prefix = "fare")
Embarked_dum2 = pd.get_dummies(test['Embarked'], prefix = "embarked")
Pclass_dum = pd.get_dummies(test['Pclass'], prefix = "pclass")

#Añadimos las nuevas variables, ahora como dummies, al dataset
test = pd.concat([test, Embarked_dum2, Age_dum2, Fare_dum2, Pclass_dum], axis = 1)
#Se borran también algunos dummies para asegurar invertibilidad de la matríz 
test = test.drop(columns = ['Pclass','Age', 'Fare', 'Embarked',"Age_cat","age_adult", "Fare_cat","embarked_C","fare_expensive","pclass_1"])
test.head()

In [ ]:
train = train[['Sex', 'age_kid','SibSp','Parch','embarked_S','embarked_Q','fare_cheap','pclass_2','pclass_3','Cabin','Survived']]
train.head()

In [ ]:
test = test[['Sex', 'age_kid','SibSp','Parch','embarked_S','embarked_Q','fare_cheap','pclass_2','pclass_3','Cabin']]
test.head()

## 2.- Entrenamiento de Modelo

### 2.1.- Datasets de Entrenamiento y Validación

##### Dataframe de Datos

In [ ]:
train_data = train.iloc[:,0:len(train.columns)-1]

##### Dataframe de Variable Objetivo

In [ ]:
train_target = train.iloc[:,len(train.columns)-1]  

##### Lista de Características de los Datos

In [ ]:
features = train_data.columns.values
features

##### Separación de Dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_target,test_size=0.3,random_state=12)

##### Normalización de los Datos

In [ ]:
# from sklearn.preprocessing import StandardScaler 
#scaler = StandardScaler()
scaler = MinMaxScaler()
scaler.fit(train_data)
scaler.fit(test)

test = scaler.transform(test)
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

### 2.2.-Entrenamiento del Modelo

##### Selección de Modelo Secuencial de Keras

In [ ]:
model = models.Sequential()
best_test_accuracy = 0

##### Modelo 1  de Red Neuronal
Se escoje un momedelo inicial de 3 capas con las siguientes características:

- learning rate = 0.01
- Capa 1 = 'relu', 20 nodos
- Capa 2 = 'relu', 20 nodos
- Capa 3 = 'sigmoid', 1 nodo

In [ ]:
model = models.Sequential()

# Capa 1
model.add(Dense(20, kernel_initializer = 'uniform', activation='relu', input_dim=len(features)))
# Capa 2
model.add(Dense(20, kernel_initializer = 'uniform', activation='relu'))
# Capa de Salida
model.add(Dense(1, kernel_initializer = 'uniform', activation='sigmoid'))

In [ ]:
# define our loss function and optimizer
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])

# train the parameters
history = model.fit(X_train, y_train, epochs=20, batch_size=16)

# evaluate accuracy
train_acc = model.evaluate(X_train, y_train, batch_size=16)[1]
test_acc = model.evaluate(X_test, y_test, batch_size=16)[1]
print('Training accuracy: %s' % train_acc)
print('Testing accuracy: %s' % test_acc)

losses = history.history['loss']

plt.plot(range(len(losses)), losses, 'r')
plt.show()

In [ ]:
if best_test_accuracy < test_acc:
    best_test_accuracy = test_acc
    model_best = model

print(f'MODELO:\n-------------------------------------------------------------------')
model.summary()
i=0
for layer in model.layers:
    i=i+1
    print(f'Función Activación Capa {i} = {layer.activation}')
print(f'\nPrecisión:{test_acc}' ) 

##### Modelo 2 de Red Neuronal
- learning rate = 0.008
- Capa 1 = 'relu', 20 nodos
- Capa 2 = 'relu', 20 nodos
- Capa 3 = 'relu', 10 nodos
- Capa 4 = 'relu', 20 nodos
- Capa 5 = 'relu', 10 nodos
- Capa 6 = 'sigmoid'

In [ ]:
model = models.Sequential()

# Capa 1
model.add(Dense(20, kernel_initializer = 'uniform', activation='relu', input_dim=len(features)))
# Capa 2
model.add(Dense(20, kernel_initializer = 'uniform', activation='relu'))
# Capa 3
model.add(Dense(10, kernel_initializer = 'uniform', activation='relu'))
# Capa 4
model.add(Dense(20, kernel_initializer = 'uniform', activation='relu'))
# Capa 5
model.add(Dense(10, kernel_initializer = 'uniform', activation='relu'))
# Capa de Salida
model.add(Dense(1, kernel_initializer = 'uniform', activation='sigmoid'))

In [ ]:
# define our loss function and optimizer
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.008), metrics=['accuracy'])

# train the parameters
history = model.fit(X_train, y_train, epochs=20, batch_size=16)

# evaluate accuracy
train_acc = model.evaluate(X_train, y_train, batch_size=16)[1]
test_acc = model.evaluate(X_test, y_test, batch_size=16)[1]
print('Training accuracy: %s' % train_acc)
print('Testing accuracy: %s' % test_acc)

losses = history.history['loss']

plt.plot(range(len(losses)), losses, 'r')
plt.show()

In [ ]:
if best_test_accuracy < test_acc:
    best_test_accuracy = test_acc
    model_best = model

print(f'MODELO:\n-------------------------------------------------------------------')
model.summary()
i=0
for layer in model.layers:
    i=i+1
    print(f'Función Activación Capa {i} = {layer.activation}')
print(f'\nPrecisión:{test_acc}' ) 

##### Modelo 3  de Red Neuronal
- learning rate = 0.01
- Capa 1 = 'relu'
- Capa 2 = 'sigmoid'
- Capa 3 = 'sigmoid'

In [ ]:
model = models.Sequential()

# Capa 1
model.add(Dense(20, kernel_initializer = 'uniform', activation='relu', input_dim=len(features)))
# Capa 2
model.add(Dense(20, kernel_initializer = 'uniform', activation='sigmoid'))
# Capa de Salida
model.add(Dense(1, kernel_initializer = 'uniform', activation='sigmoid'))

In [ ]:
# define our loss function and optimizer
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])

# train the parameters
history = model.fit(X_train, y_train, epochs=20, batch_size=16)

# evaluate accuracy
train_acc = model.evaluate(X_train, y_train, batch_size=16)[1]
test_acc = model.evaluate(X_test, y_test, batch_size=16)[1]
print('Training accuracy: %s' % train_acc)
print('Testing accuracy: %s' % test_acc)

losses = history.history['loss']

plt.plot(range(len(losses)), losses, 'r')
plt.show()

In [ ]:
if best_test_accuracy < test_acc:
    best_test_accuracy = test_acc
    model_best = model

print(f'MODELO:\n-------------------------------------------------------------------')
model.summary()
i=0
for layer in model.layers:
    i=i+1
    print(f'Función Activación Capa {i} = {layer.activation}')
print(f'\nPrecisión:{test_acc}' ) 

##### Modelo 4 de Red Neuronal
- learning rate = 0.009
- Capa 1 = 'relu', 20 nodos
- Capa 2 = 'relu', 20 nodos
- Capa 3 = 'relu', 10 nodos
- Capa 4 = 'relu', 20 nodos
- Capa 5 = 'relu', 10 nodos
- Capa 6 = 'sigmoid'

In [ ]:
model = models.Sequential()

# Capa 1
model.add(Dense(20, kernel_initializer = 'uniform', activation='relu', input_dim=len(features)))
# Capa 2
model.add(Dense(20, kernel_initializer = 'uniform', activation='relu'))
# Capa 3
model.add(Dense(10, kernel_initializer = 'uniform', activation='relu'))
# Capa 4
model.add(Dense(20, kernel_initializer = 'uniform', activation='relu'))
# Capa 5
model.add(Dense(10, kernel_initializer = 'uniform', activation='relu'))
# Capa de Salida
model.add(Dense(1, kernel_initializer = 'uniform', activation='sigmoid'))

In [ ]:
# define our loss function and optimizer
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.009), metrics=['accuracy'])

# train the parameters
history = model.fit(X_train, y_train, epochs=20, batch_size=16)

# evaluate accuracy
train_acc = model.evaluate(X_train, y_train, batch_size=16)[1]
test_acc = model.evaluate(X_test, y_test, batch_size=16)[1]
print('Training accuracy: %s' % train_acc)
print('Testing accuracy: %s' % test_acc)

losses = history.history['loss']

plt.plot(range(len(losses)), losses, 'r')
plt.show()

In [ ]:
if best_test_accuracy < test_acc:
    best_test_accuracy = test_acc
    model_best = model

print(f'MODELO:\n-------------------------------------------------------------------')
model.summary()
i=0
for layer in model.layers:
    i=i+1
    print(f'Función Activación Capa {i} = {layer.activation}')
print(f'\nPrecisión:{test_acc}' ) 

## 3.- Pruebas de Modelo

De acuerdo a las pruebas realizadas con variados modelos con diferentes características y ajustes de parámetros, hemos optado por considerar cuatro modelos de diferentes caracteristicas( learning rate, cantidad de capas, función de activación y cantidad de nodos).

En base a lo anterior, los criterios utilizados para realizar el ajuste y selección de los modelos fue de acuerdo a los siguientes criterios:
 
 ##### 1.- Al aplicar una función de activación a “sigmoid” estamos proyectando una salida que sea de 0 a 1 y eso nos ayudaría de mejor manera a clasificar ya sea si sobrevive (1) o no sobrevive(0).
##### 2.- El optimizador Adam combina las bondades de Adagrad y RMSprop este algoritmo que es mas reciente que los anteriores y esta construido en base a sus predecesores se esperaria que su rendimiento sea superior.
##### 3.- Para las funciones de coste se utilizara binary cross entropy para el clasificador binario.

In [ ]:
print(f'MEJOR MODELO:\n-------------------------------------------------------------------')
model_best.summary()
i=0
for layer in model_best.layers:
    i=i+1
    print(f'Función Activación Capa {i} = {layer.activation}')
print(f'\nPrecisión:{best_test_accuracy}' ) 

In [ ]:
predict_x=((model_best.predict(test) > 0.5).astype('int32'))


In [ ]:
type(test)

In [ ]:
test_survived = pd.DataFrame(predict_x, columns=['Survived'])

In [ ]:
test_survived.value_counts()

In [ ]:
test_df = pd.DataFrame(test, columns= ['Sex', 'age_kid','SibSp','Parch','embarked_S','embarked_Q','fare_cheap','pclass_2','pclass_3','Cabin'])

In [ ]:
test_df.head()

In [ ]:
test_complete = pd.concat([test_df, test_survived], axis=1)

In [ ]:
test_complete

In [ ]:
data = test_complete
data

In [ ]:
# Se preparan las visualizaciones 
fig, axes = plt.subplots(nrows=2, ncols=2, figsize = (12,8))
fig.subplots_adjust(hspace=0.3)
axes = axes.flatten()
ax = axes[0]
# Distribución de los datos según sexo
sns.countplot(x='Sex', data=data, palette=['#73DAF5','#F5C5F5'],ax=ax)
ax.set_xticklabels(['Hombre','Mujer'])
ax.set_title('b) Distribución por Sexo')
ax = axes[2]
# Revisamos que tan balanceadas estan nuestras clases
sns.countplot(x='Survived', data=data, palette=['#F57373','#8FF573'],ax=ax)
ax.set_title('c) Distribución de variable Target')
ax.set_xticklabels(['No sobrevive','Sobrevive'],rotation=0)
ax = axes[3]
#Frecuencia de diagnóstico por sexo
pd.crosstab(test_complete.Sex,test_complete.Survived).plot(kind="bar",ax=ax,color=['#8FF573','#F57373'])
ax.set_title('d) Frecuencia de diagnóstico por Sexo')
ax.set_xticklabels(['Hombre','Mujer'],rotation=0)
ax.legend(["Sobrevive", "No sobrevive"])
ax.set_ylabel('Frecuencia')

## 3.1.- Resultado Predictivo Para Los Miembros del Equipo

Se ingresan los valores señalados por cada miembro del grupo, mas un 'dummy' para poder ejecutar de manera correcta la estandarizacion de los datos

In [ ]:
Patricio = {'Sex':1, 'age_kid':0, 'SibSp':0, 'Parch':0, 'embarked_S':1, 'embarked_Q':0, 'fare_cheap':1, 'pclass_2':1, 'pclass_3':0, 'Cabin':1}
Carlos = {'Sex':1, 'age_kid':0, 'SibSp':0, 'Parch':0, 'embarked_S':0, 'embarked_Q':0, 'fare_cheap':0, 'pclass_2':0, 'pclass_3':0, 'Cabin':1}
Nicolas = {'Sex':1, 'age_kid':0, 'SibSp':0, 'Parch':0, 'embarked_S':0, 'embarked_Q':1, 'fare_cheap':0, 'pclass_2':1, 'pclass_3':0, 'Cabin':1}
Pablo = {'Sex':1, 'age_kid':0, 'SibSp':0, 'Parch':0, 'embarked_S':1, 'embarked_Q':0, 'fare_cheap':0, 'pclass_2':0, 'pclass_3':0, 'Cabin':1}
Dummy = {'Sex':0, 'age_kid':0, 'SibSp':0, 'Parch':0, 'embarked_S':1, 'embarked_Q':0, 'fare_cheap':0, 'pclass_2':0, 'pclass_3':0, 'Cabin':0}


In [ ]:
Grupo_predict = pd.DataFrame(columns=['Sex', 'age_kid','SibSp','Parch','embarked_S','embarked_Q','fare_cheap','pclass_2','pclass_3','Cabin'])

In [ ]:
Grupo_predict = Grupo_predict.append([Patricio, Carlos, Nicolas, Pablo, Dummy], ignore_index=True)

In [ ]:
Grupo_predict

In [ ]:
Grupo_predict = Grupo_predict.astype(int)
Grupo_predict.info()

In [ ]:
scaler = MinMaxScaler()
scaler.fit(Grupo_predict)

Grupo_predict = scaler.transform(Grupo_predict)

In [ ]:
Grupo_predict

In [ ]:
predict_grupo=((model_best.predict(Grupo_predict) > 0.5).astype('int32'))

In [ ]:
predict_grupo

In [ ]:
Grupo_predicted = pd.DataFrame(Grupo_predict, columns= ['Sex', 'age_kid','SibSp','Parch','embarked_S','embarked_Q','fare_cheap','pclass_2','pclass_3','Cabin'])

Grupo_predicted['Nombres'] = ['Patricio', 'Carlos', 'Nicolas', 'Pablo', 'Dummy']

Grupo_predicted = Grupo_predicted.drop(Grupo_predicted.index[4])

In [ ]:
Grupo_predicted

In [ ]:
Grupo_survived = pd.DataFrame(predict_grupo, columns=['Survived'])
Grupo_complete = pd.concat([Grupo_predicted, Grupo_survived], axis=1)

In [ ]:
Grupo_complete = Grupo_complete.drop(Grupo_complete.index[4])

In [ ]:
Grupo_complete

### Dado que tres de cuatro hombres sobreviven, nos damos cuenta de que el valor que tenemos como threshold para el predict de 0.5 hace que algunos de nosotros sobrevivan, siendo que lo mas probable es que todos estabamos un poco mas arriba que 0.5. Mientras que la persona que murio estaba un poco mas abajo del 0.5

### Cuando se usa un umbral de 0.8, ya nadie sobrevive.

## 4.- Conclusion

### En conclusion, se logra dar un contexto general de la data, hacer una limpieza y visualizacion la cual hace que la data se pueda entrenar para asi lograr el objetivo de la tarea la cual es implementar un modelo de red neuronal y ver quienes sobreviven y quienes no. Asi, logramos aprender e identificar los distintos parametros los cuales componen al modelo secuencial, como la funcion de activacion, el learning rate, el optimizador, las distintas capas, el numero de neuronas y la funcion de costo.

### Por ultimo, logramos identificar los mejores modelos para despues utilizarlos en la prediccion de los datos de prueba y para los datos del grupo.